In [ ]:
!nvcc --version
!nvidia-smi

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0
Mon Nov 17 08:12:06 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   41C    P8       

In [ ]:
%%writefile cuda_analysis.cu

Overwriting cuda_analysis.cu


In [ ]:
%%writefile cuda_analysis.cu
// ----------------------------------------------------------------------
// Proyecto 2 micro
// Parte CUDA
// cuda_analysis.cu
// Autores: Pedro Caso, Diego Calderon, Hugo Mendez y Arodi Chavez
// ----------------------------------------------------------------------

#include <iostream>
#include <fstream>
#include <sstream>
#include <vector>
#include <string>
#include <cmath>
#include <thrust/device_vector.h>
#include <thrust/host_vector.h>
#include <thrust/transform.h>
#include <thrust/reduce.h>
#include <thrust/execution_policy.h>
#include <thrust/functional.h>
#include <thrust/copy.h>

using namespace std;

// Constantes
const double SAMPLE_INTERVAL = 10.0;

const double P0 = 101325.0;
const double H_SCALE = 8434.0;
const double T0_K = 288.15;
const double L_GRA = 0.0065;

// Estructura CSV
struct Record {
    string timestamp;
    double presion_hpa;
    double temp_motor_c;
    double luminosidad_raw;
    double distancia_cm;
    int estado;
};

static vector<string> splitCSVLine(const string &line) {
    vector<string> out;
    string cur;
    for (char c : line) {
        if (c == ',') {
            out.push_back(cur);
            cur.clear();
        } else {
            cur.push_back(c);
        }
    }
    out.push_back(cur);
    return out;
}

vector<Record> readCSV(const string &path) {
    ifstream f(path);
    vector<Record> data;
    if (!f.is_open()) {
        cerr << "No se pudo abrir " << path << endl;
        return data;
    }
    string line;
    getline(f, line);
    while (getline(f, line)) {
        if (line.size() <= 1) continue;
        auto c = splitCSVLine(line);
        if (c.size() < 6) continue;
        Record r;
        r.timestamp = c[0];
        r.presion_hpa = stod(c[1]);
        r.temp_motor_c = stod(c[2]);
        r.luminosidad_raw = stod(c[3]);
        r.distancia_cm = stod(c[4]);
        r.estado = stoi(c[5]);
        data.push_back(r);
    }
    return data;
}

// Función de Soporte para GPU/CPU
__host__ __device__
double estimate_ambient_temp_c_from_pressure_hpa(double presion_hpa) {
    double p = presion_hpa * 100.0;
    if (p <= 0) return 15.0;
    double ratio = p / P0;
    if (ratio <= 0) return 15.0;

    double h = -H_SCALE * log(ratio);
    double T_amb_K = T0_K - L_GRA * h;
    return T_amb_K - 273.15;
}

// KERNEL 1: Calculo de ΔT (TPI)
__global__
void compute_deltaT_kernel(const double* temp_motor,
                           const double* presion_hpa,
                           double* deltaT,
                           int n)
{
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx >= n) return;

    double Tamb = estimate_ambient_temp_c_from_pressure_hpa(presion_hpa[idx]);
    deltaT[idx] = temp_motor[idx] - Tamb;
}

// KERNEL 2: analisis de eventos
__global__
void mark_flags_kernel(const double* luminosidad,
                       const int* estado,
                       int n,
                       unsigned char* headlight_flag,
                       unsigned char* night_flag,
                       unsigned char* headlight_rising,
                       unsigned char* stop_event,
                       unsigned char* start_event)
{
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx >= n) return;


    // Faros encendidos
    unsigned char hf = (luminosidad[idx] == 1.0 && estado[idx] == 1) ? 1 : 0;

    // cuando esta oscuro
    unsigned char nf = (luminosidad[idx] == 1.0) ? 1 : 0;

    headlight_flag[idx] = hf;
    night_flag[idx] = nf;

    // Detectar inicio de encendido de faros
    unsigned char prev = 0;
    if (idx > 0) {
        unsigned char prev_h = (luminosidad[idx-1] == 1.0 && estado[idx-1] == 1) ? 1 : 0;
        prev = prev_h;
    }
    headlight_rising[idx] = (hf && !prev) ? 1 : 0;

    // Eventos de estado
    int s = estado[idx];
    int sp = (idx > 0) ? estado[idx-1] : 0;

    stop_event[idx] = (s == 0 && sp == 1) ? 1 : 0;
    start_event[idx]  = (s == 1 && sp == 0) ? 1 : 0;
}

// KERNEL 3: Estadisticas HC-SR04 (Retroceso)
__global__
void hc_masked_sum_kernel(const double* dist,
                          const int* estado,
                          int n,
                          double* sum_out,
                          double* sumsq_out,
                          int* count_out)
{
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx >= n) return;

    double v = dist[idx];
    atomicAdd(sum_out, v);
    atomicAdd(sumsq_out, v*v);
    atomicAdd(count_out, 1);

}

// Main
int main(int argc, char** argv) {

    if (argc < 2) {
        cerr << "Uso: " << argv[0] << " archivo.csv\n";
        return 1;
    }

    cout << "SISTEMA CARRO INTELIGENTE - ANÁLISIS CUDA \n" << endl;

    auto rec = readCSV(argv[1]);
    size_t N = rec.size();
    if (N == 0) {
        cerr << "No se leyeron datos\n";
        return 1;
    }

    cout << "Registros leidos: " << N << endl;

    // Extraer datos a vectores
    vector<double> temp(N), pres(N), lum(N), dist(N);
    vector<int> est(N);
    vector<string> time(N);

    for (size_t i=0; i<N; i++) {
        time[i] = rec[i].timestamp;
        pres[i] = rec[i].presion_hpa;
        temp[i] = rec[i].temp_motor_c;
        lum[i]  = rec[i].luminosidad_raw;
        dist[i] = rec[i].distancia_cm;
        est[i]  = rec[i].estado;
    }

    // Transferir a GPU
    thrust::device_vector<double> d_temp = temp;
    thrust::device_vector<double> d_pres = pres;
    thrust::device_vector<double> d_lum  = lum;
    thrust::device_vector<double> d_dist = dist;
    thrust::device_vector<int>    d_est  = est;

    thrust::device_vector<double> d_deltaT(N);
    thrust::device_vector<unsigned char> d_hf(N), d_nf(N), d_hr(N), d_ue(N), d_se(N);

    cout << "Datos transferidos a GPU\n" << endl;

    // KERNEL 1: TPI (ΔT)
    cout << "Ejecutando KERNEL 1: indice de rendimiento termico (TPI)..." << endl;
    {
        int threads = 256;
        int blocks = (N + threads - 1)/threads;
        compute_deltaT_kernel<<<blocks, threads>>>(
            thrust::raw_pointer_cast(d_temp.data()),
            thrust::raw_pointer_cast(d_pres.data()),
            thrust::raw_pointer_cast(d_deltaT.data()),
            N
        );
        cudaDeviceSynchronize();
        cout << "ΔT calculado para " << N << " registros\n" << endl;
    }

    // KERNEL 2: procesamiento de Eventos
    cout << "Ejecutando KERNEL 2: procesamiento de Eventos (Faros, desbloqueo, encendido)..." << endl;
    {
        int threads = 256;
        int blocks = (N + threads - 1)/threads;
        mark_flags_kernel<<<blocks, threads>>>(
            thrust::raw_pointer_cast(d_lum.data()),
            thrust::raw_pointer_cast(d_est.data()),
            N,
            thrust::raw_pointer_cast(d_hf.data()),
            thrust::raw_pointer_cast(d_nf.data()),
            thrust::raw_pointer_cast(d_hr.data()),
            thrust::raw_pointer_cast(d_ue.data()),
            thrust::raw_pointer_cast(d_se.data())
        );
        cudaDeviceSynchronize();
        cout << "Eventos marcados correctamente\n" << endl;
    }

    // KERNEL 3: Análisis TPI con Thrust
    cout << "Ejecutando KERNEL 3: Análisis Estadístico TPI (Thrust Reduce)..." << endl;
    double sumDelta = thrust::reduce(d_deltaT.begin(), d_deltaT.end(),
                                      0.0, thrust::plus<double>());
    double avgDelta = sumDelta / (double)N;
    double maxDelta = thrust::reduce(d_deltaT.begin(), d_deltaT.end(),
                                      -1e308, thrust::maximum<double>());
    cout << "ΔT Promedio: " << avgDelta << " °C" << endl;
    cout << "ΔT Máximo: " << maxDelta << " °C\n" << endl;


    // KERNEL 4: Analisis de Faros
    cout << "Ejecutando KERNEL 4: Análisis de Duración de Faros..." << endl;
    long total_on = thrust::reduce(d_hf.begin(), d_hf.end(),
                                    (long)0, thrust::plus<long>());
    long total_events = thrust::reduce(d_hr.begin(), d_hr.end(),
                                        (long)0, thrust::plus<long>());

    double total_headlight_time = total_on * SAMPLE_INTERVAL;
    double avg_headlight_duration = (total_events > 0)
        ? total_headlight_time / total_events
        : 0.0;

    cout << "Tiempo total faros encendidos: " << total_headlight_time << " s" << endl;
    cout << "Eventos de encendido de faros: " << total_events << endl;
    cout << "Duracion promedio por evento: " << avg_headlight_duration << " s\n" << endl;


    // KERNEL 5: Analisis HC-SR04
    cout << "Ejecutando KERNEL 5: Análisis Estadístico de asistencia en retroceso (HC-SR04)..." << endl;

    double *d_sum, *d_sumsq;
    int *d_count;

    cudaMalloc(&d_sum, sizeof(double));
    cudaMalloc(&d_sumsq, sizeof(double));
    cudaMalloc(&d_count, sizeof(int));

    cudaMemset(d_sum, 0, sizeof(double));
    cudaMemset(d_sumsq, 0, sizeof(double));
    cudaMemset(d_count, 0, sizeof(int));

    {
        int threads = 256;
        int blocks = (N + threads - 1)/threads;
        hc_masked_sum_kernel<<<blocks, threads>>>(
            thrust::raw_pointer_cast(d_dist.data()),
            thrust::raw_pointer_cast(d_est.data()),
            N,
            d_sum, d_sumsq, d_count
        );
        cudaDeviceSynchronize();
    }


    double sum_hc, sumsq_hc;
    int count_hc;

    cudaMemcpy(&sum_hc,  d_sum,   sizeof(double), cudaMemcpyDeviceToHost);
    cudaMemcpy(&sumsq_hc, d_sumsq, sizeof(double), cudaMemcpyDeviceToHost);
    cudaMemcpy(&count_hc, d_count, sizeof(int),    cudaMemcpyDeviceToHost);

    cudaFree(d_sum);
    cudaFree(d_sumsq);
    cudaFree(d_count);

    // Procesamiento estadistico en CPU
    double mean_hc = (count_hc > 0 ? sum_hc / count_hc : 0.0);
    double stddev_hc = 0.0;
    if (count_hc > 0) {
        double var = (sumsq_hc / count_hc) - (mean_hc * mean_hc);
        stddev_hc = (var > 0 ? sqrt(var) : 0.0);
    }

    cout << "Mediciones en Retroceso: " << count_hc << endl;
    cout << "Distancia promedio: " << mean_hc << " cm" << endl;
    cout << "Desviación estándar: " << stddev_hc << " cm\n" << endl;

    // Análisis Frecuencia de Uso
    cout << "Calculando Análisis de Frecuencia de Uso..." << endl;

    vector<unsigned char> v_ue(N), v_se(N);
    thrust::copy(d_ue.begin(), d_ue.end(), v_ue.begin());
    thrust::copy(d_se.begin(), d_se.end(), v_se.begin());


    vector<size_t> idxOFF, idxON;
    for (size_t i=0; i<N; i++) {
        if (v_ue[i]) idxOFF.push_back(i);  // Momento donde se apago
        if (v_se[i]) idxON.push_back(i);   // Momento donde se encendio
    }

    // Calcular intervalos: tiempo que el carro estuvo ENCENDIDO
    vector<double> intervals_on;
    size_t j = 0;
    for (size_t i=0; i<idxON.size(); i++) {
        size_t on_idx = idxON[i];

        // Buscar el siguiente apagado después de este encendido
        while (j < idxOFF.size() && idxOFF[j] <= on_idx) j++;

        if (j < idxOFF.size()) {
            double dt = (idxOFF[j] - on_idx) * SAMPLE_INTERVAL;
            intervals_on.push_back(dt);
            j++;
        }
    }

    double mean_on_duration = 0.0;
    if (!intervals_on.empty()) {
        double acc = 0;
        for (double v : intervals_on) acc += v;
        mean_on_duration = acc / intervals_on.size();
    }

    cout << "Eventos apagado: " << idxOFF.size() << endl;
    cout << "Eventos encendido: " << idxON.size() << endl;
    cout << "Tiempo promedio encendido (por ciclo): " << mean_on_duration << " s\n" << endl;

    // Guardar Resultados en CSV
    cout << "Guardando resultados..." << endl;

    ofstream out("resumen_resultados_cuda.csv");
    out << "medicion,valor\n";
    out << "N," << N << "\n";
    out << "deltaT_promedio_C," << avgDelta << "\n";
    out << "deltaT_maximo_C," << maxDelta << "\n";
    out << "tiempo_total_faros_encendidos_s," << total_headlight_time << "\n";
    out << "eventos_faros," << total_events << "\n";
    out << "duracion_promedio_faros_s," << avg_headlight_duration << "\n";
    out << "mediciones_hcsr04_retroceso," << count_hc << "\n";
    out << "distancia_promedio_cm," << mean_hc << "\n";
    out << "desviacion_estandar_cm," << stddev_hc << "\n";
    out << "eventos_apagado," << idxOFF.size() << "\n";
    out << "eventos_encendido," << idxON.size() << "\n";
    out << "ciclos_completos," << intervals_on.size() << "\n";
    out << "Tpromedio_encendido_s," << mean_on_duration << "\n";
    out.close();

    vector<double> host_deltaT(N);
    thrust::copy(d_deltaT.begin(), d_deltaT.end(), host_deltaT.begin());

    ofstream hist("deltaT_historial.csv");
    hist << "Timestamp,DeltaT_C,TempMotor_C,TempAmbienteEstimada_C\n";
    for (size_t i=0; i<N; i++) {
        double Tamb = estimate_ambient_temp_c_from_pressure_hpa(pres[i]);
        hist << time[i] << ","
             << host_deltaT[i] << ","
             << temp[i] << ","
             << Tamb << "\n";
    }
    hist.close();

    cout << "\nANALISIS COMPLETADO\n";
    cout << "resumen_resultados_cuda.csv" << endl;
    cout << "deltaT_historial.csv" << endl;

    return 0;
}

Overwriting cuda_analysis.cu


In [ ]:
!nvcc -arch=sm_75 -O2 cuda_analysis.cu -o cuda_analysis

In [ ]:
!./cuda_analysis datos_simulados.csv

SISTEMA CARRO INTELIGENTE - ANÁLISIS CUDA 

Registros leidos: 5000
Datos transferidos a GPU

Ejecutando KERNEL 1: indice de rendimiento termico (TPI)...
ΔT calculado para 5000 registros

Ejecutando KERNEL 2: procesamiento de Eventos (Faros, desbloqueo, encendido)...
Eventos marcados correctamente

Ejecutando KERNEL 3: Análisis Estadístico TPI (Thrust Reduce)...
ΔT Promedio: 23.3691 °C
ΔT Máximo: 31.417 °C

Ejecutando KERNEL 4: Análisis de Duración de Faros...
Tiempo total faros encendidos: 29810 s
Eventos de encendido de faros: 1181
Duracion promedio por evento: 25.2413 s

Ejecutando KERNEL 5: Análisis Estadístico de asistencia en retroceso (HC-SR04)...
Mediciones en Retroceso: 5000
Distancia promedio: 63.3301 cm
Desviación estándar: 29.6716 cm

Calculando Análisis de Frecuencia de Uso...
Eventos apagado: 1181
Eventos encendido: 1181
Tiempo promedio encendido (por ciclo): 25.2413 s

Guardando resultados...

ANALISIS COMPLETADO
resumen_resultados_cuda.csv
deltaT_historial.csv


In [ ]:
import pandas as pd

print("RESUMEN DE RESULTADOS\n")
try:
    results = pd.read_csv('resumen_resultados_cuda.csv')
    print(results.to_string(index=False))
except FileNotFoundError:
    print("No se genero resumen_resultados_cuda.csv")

print("\n" + "-"*60)
print("PRIMERAS 10 FILAS DE HISTORIAL ΔT\n")
try:
    history = pd.read_csv('deltaT_historial.csv')
    print(history.head(10).to_string(index=False))
except FileNotFoundError:
    print("No se genero deltaT_historial.csv")

RESUMEN DE RESULTADOS

                       medicion      valor
                              N  5000.0000
              deltaT_promedio_C    23.3691
                deltaT_maximo_C    31.4170
tiempo_total_faros_encendidos_s 29810.0000
                  eventos_faros  1181.0000
      duracion_promedio_faros_s    25.2413
    mediciones_hcsr04_retroceso  5000.0000
          distancia_promedio_cm    63.3301
         desviacion_estandar_cm    29.6716
                eventos_apagado  1181.0000
              eventos_encendido  1181.0000
               ciclos_completos  1181.0000
          Tpromedio_encendido_s    25.2413

------------------------------------------------------------
PRIMERAS 10 FILAS DE HISTORIAL ΔT

                 Timestamp  DeltaT_C  TempMotor_C  TempAmbienteEstimada_C
2025-11-17 00:31:58.322145   29.2458        38.52                 9.27420
2025-11-17 00:32:08.322145   25.1454        33.92                 8.77463
2025-11-17 00:32:18.322145   19.7112        28.41       